<a href="https://colab.research.google.com/github/markusloecher/DataScience2021/blob/main/TWSM/Fine_tuning_a_model_with_Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine-tuning a model with Keras

Install the Transformers and Datasets libraries to run this notebook.

In [1]:
!pip install datasets transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 8.2 MB/s 
     |████████████████████████████████| 4.2 MB 68.7 MB/s 
     |████████████████████████████████| 86 kB 7.1 MB/s 
     |████████████████████████████████| 212 kB 62.9 MB/s 
     |████████████████████████████████| 86 kB 6.3 MB/s 
     |████████████████████████████████| 140 kB 62.1 MB/s 
     |████████████████████████████████| 1.1 MB 55.5 MB/s 
     |████████████████████████████████| 596 kB 59.6 MB/s 
     |████████████████████████████████| 127 kB 79.5 MB/s 
     |████████████████████████████████| 144 kB 72.4 MB/s 
     |████████████████████████████████| 271 kB 67.4 MB/s 
     |████████████████████████████████| 94 kB 4.2 MB/s 
     |████████████████████████████████| 6.6 MB 57.1 MB/s 
     |████████████████████████████████| 1.2 MB 60.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd
dev = pd.read_csv("/content/drive/Othercomputers/Mi portátil/Master/GitHub/twsm-PERSONAL/Data/QQP/dev.tsv", sep = "\t")
train = pd.read_csv("/content/drive/Othercomputers/Mi portátil/Master/GitHub/twsm-PERSONAL/Data/QQP/train.tsv", sep = "\t")
test = pd.read_csv("/content/drive/Othercomputers/Mi portátil/Master/GitHub/twsm-PERSONAL/Data/QQP/test.tsv", sep = "\t")

In [7]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
import numpy as np

raw_datasets = load_dataset("glue", "qqp")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Reusing dataset glue (/root/.cache/huggingface/datasets/glue/qqp/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
raw_datasets = load_dataset("glue","qqp", split = "train[:25%]") #this could also all be one line with splits in a list
raw_val = load_dataset("glue","qqp", split = "validation[:50%]")
raw_test = load_dataset("glue","qqp", split = "test[:50%]")

Reusing dataset glue (/root/.cache/huggingface/datasets/glue/qqp/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Reusing dataset glue (/root/.cache/huggingface/datasets/glue/qqp/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Reusing dataset glue (/root/.cache/huggingface/datasets/glue/qqp/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


In [10]:
raw_datasets, raw_val, raw_test

(Dataset({
     features: ['question1', 'question2', 'label', 'idx'],
     num_rows: 90962
 }), Dataset({
     features: ['question1', 'question2', 'label', 'idx'],
     num_rows: 20215
 }), Dataset({
     features: ['question1', 'question2', 'label', 'idx'],
     num_rows: 195482
 }))

In [12]:
def tokenize_function(example):
    return tokenizer(example["question1"], example["question2"], truncation=True)


tokenized_train = raw_datasets.map(tokenize_function, batched=True)
tokenized_val = raw_val.map(tokenize_function, batched=True)
tokenized_test = raw_test.map(tokenize_function, batched = True)

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

tf_train_dataset = tokenized_datasets.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_validation_dataset = tokenized_val.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

tf_test_dataset = tokenized_test.to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["labels"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

  0%|          | 0/91 [00:00<?, ?ba/s]

  0%|          | 0/21 [00:00<?, ?ba/s]

  0%|          | 0/196 [00:00<?, ?ba/s]

In [13]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Downloading:   0%|          | 0.00/511M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy

model.compile(
    optimizer="adam",
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)
model.fit(
    tf_train_dataset,
    validation_data=tf_validation_dataset,
)

11370/11370 [==============================] - 1503s 131ms/step - loss: 0.6855 - accuracy: 0.6046 - val_loss: 0.6570 - val_accuracy: 0.6340


In [15]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay

batch_size = 120
num_epochs = 3
# The number of training steps is the number of samples in the dataset, divided by the batch size then multiplied
# by the total number of epochs. Note that the tf_train_dataset here is a batched tf.data.Dataset,
# not the original Hugging Face Dataset, so its len() is already num_samples // batch_size.
num_train_steps = len(tf_train_dataset) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)
from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=lr_scheduler)

In [16]:
import tensorflow as tf

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=3)

Epoch 1/3
11370/11370 [==============================] - 1496s 130ms/step - loss: 0.3804 - accuracy: 0.8233 - val_loss: 0.3186 - val_accuracy: 0.8555
Epoch 2/3
11370/11370 [==============================] - 1491s 131ms/step - loss: 0.2339 - accuracy: 0.9036 - val_loss: 0.3059 - val_accuracy: 0.8749
Epoch 3/3
11370/11370 [==============================] - 1488s 131ms/step - loss: 0.1082 - accuracy: 0.9611 - val_loss: 0.3916 - val_accuracy: 0.8750


In [18]:
preds = model.predict(tf_validation_dataset)["logits"]

In [19]:
class_preds = np.argmax(preds, axis=1)
print(preds.shape, class_preds.shape)

(20208, 2) (20208,)


In [20]:
from datasets import load_metric

metric = load_metric("glue", "qqp")
metric.compute(predictions=class_preds, references=raw_datasets["validation"]["label"])

KeyError: ignored